In [1]:
import sys
import dotenv
dotenv.load_dotenv('../../.env')

sys.path.insert(0, '../')
sys.path.insert(0, '../app')

from app.processing.audio.SIWhisperModel import SIWhisperModel
from app.SIWeaviateClient import SIWeaviateClient
from app.processing.YoutubeProcessor import YoutubeProcessor

whisper = SIWhisperModel('medium', 'whisper-medium')



/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/../app/schemas.py:121: GenericBeforeBaseModelWarning: Classes should inherit from `BaseModel` before generic classes (e.g. `typing.Generic[T]`) for pydantic generics to work properly.
  class ChunkWithScore(Generic[ChunkType], BaseDocumentChunk):
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
from app.S3Storage import S3Storage


urls = [
    # "https://www.youtube.com/watch?v=re__12nH2ro",
    # "https://www.youtube.com/watch?v=FSbEorqTtMU",
    "https://www.youtube.com/watch?v=lds6ExALbjk",
    # "https://www.youtube.com/watch?v=0oB2oBj5Ex4",
    # "https://www.youtube.com/watch?v=p5JR8fsEzR4",
]

s3 = S3Storage()
with SIWeaviateClient() as client:
    for url in urls:
        YoutubeProcessor(s3=s3,client=client, whisper=whisper, youtube_url=url)

EXTRACT DOCUMNET YOUTUBE video_s3_object_name 1 youtube/3186fed9-7800-4e0d-8b26-132bbedc01e5.mp4
AUDIO CODEC aac


whisper-medium: 100%|██████████| 189.846/189.846 [00:13<00:00, 14.51s/s]           


time_taken 13.60537338256836
File /home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/documents/youtube/0b64f65b-101e-4567-9169-c2ea0dc53900.mp4 uploaded successfully to bucket science-infuse-content-dev as youtube/3186fed9-7800-4e0d-8b26-132bbedc01e5.mp4.
Public URL: https://s3.gra.io.cloud.ovh.net//science-infuse-content-dev/youtube/3186fed9-7800-4e0d-8b26-132bbedc01e5.mp4
EXTRACT DOCUMNET YOUTUBE video_s3_object_name 2 youtube/3186fed9-7800-4e0d-8b26-132bbedc01e5.mp4


# tests

In [5]:
from pytube import YouTube
yt = YouTube("https://www.youtube.com/watch?v=MP7tdlNM3pw")
video_name = yt.vid_info.get("videoDetails", {}).get("title", "Untitled Video")
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path=output_path, filename=filename)

NameError: name 'output_path' is not defined

In [35]:
from googleapiclient.discovery import build

# Set up YouTube API client
api_key = ''
youtube = build('youtube', 'v3', developerKey=api_key)

def get_channel_videos(channel_id):
    playlist_id = f"{channel_id[0]}U{channel_id[2:]}"
    videos = []
    next_page_token = None
    
    while True:
        res = youtube.playlistItems().list(
            part='snippet',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()
        
        for item in res["items"]:
            videos.append({
                "title": item["snippet"]["title"],
                "video_id": item["snippet"]["resourceId"]["videoId"],
                "publish_time": item["snippet"]["publishedAt"]
            })
        
        next_page_token = res.get('nextPageToken')
        
        if next_page_token is None:
            break
            
    return videos

videos = get_channel_videos("UC1udnO-W6gpR9qzleJ5SDKw")
print(len(videos))

442


In [25]:
videos